In [2]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size =15  # Batch size for training.
epochs = 400  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'tam.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


/home/kritz/anaconda3/envs/scripts/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Number of samples: 180
Number of unique input tokens: 53
Number of unique output tokens: 54
Max sequence length for inputs: 94
Max sequence length for outputs: 111
Train on 144 samples, validate on 36 samples
Epoch 1/400
144/144 [==============================] - 9s 65ms/step - loss: 0.8954 - val_loss: 1.3072
Epoch 2/400
144/144 [==============================] - 9s 63ms/step - loss: 0.7980 - val_loss: 1.3044
Epoch 3/400
144/144 [==============================] - 9s 61ms/step - loss: 0.7849 - val_loss: 1.2871
Epoch 4/400
144/144 [==============================] - 9s 64ms/step - loss: 0.7775 - val_loss: 1.2864
Epoch 5/400
144/144 [==============================] - 9s 65ms/step - loss: 0.7668 - val_loss: 1.2742
Epoch 6/400
144/144 [==============================] - 9s 62ms/step - loss: 0.7663 - val_loss: 1.2674
Epoch 7/400
144/144 [==============================] - 10s 72ms/step - loss: 0.7538 - val_loss: 1.2525
Epoch 8/400
144/144 [==============================] - 9s 63ms/step - loss: 

144/144 [==============================] - 11s 74ms/step - loss: 0.1621 - val_loss: 1.1500
Epoch 79/400
144/144 [==============================] - 11s 74ms/step - loss: 0.1625 - val_loss: 1.1508
Epoch 80/400
144/144 [==============================] - 11s 76ms/step - loss: 0.1534 - val_loss: 1.1712
Epoch 81/400
144/144 [==============================] - 11s 75ms/step - loss: 0.1539 - val_loss: 1.1581
Epoch 82/400
144/144 [==============================] - 11s 74ms/step - loss: 0.1471 - val_loss: 1.1816
Epoch 83/400
144/144 [==============================] - 12s 85ms/step - loss: 0.1450 - val_loss: 1.2122
Epoch 84/400
144/144 [==============================] - 14s 98ms/step - loss: 0.1411 - val_loss: 1.1888
Epoch 85/400
144/144 [==============================] - 13s 90ms/step - loss: 0.1364 - val_loss: 1.2133
Epoch 86/400
144/144 [==============================] - 12s 80ms/step - loss: 0.1303 - val_loss: 1.2293
Epoch 87/400
144/144 [==============================] - 11s 79ms/step - loss:

144/144 [==============================] - 10s 69ms/step - loss: 0.0572 - val_loss: 1.5585
Epoch 157/400
144/144 [==============================] - 10s 71ms/step - loss: 0.0569 - val_loss: 1.5596
Epoch 158/400
144/144 [==============================] - 10s 69ms/step - loss: 0.0561 - val_loss: 1.5682
Epoch 159/400
144/144 [==============================] - 10s 67ms/step - loss: 0.0570 - val_loss: 1.5654
Epoch 160/400
144/144 [==============================] - 9s 60ms/step - loss: 0.0554 - val_loss: 1.5769
Epoch 161/400
144/144 [==============================] - 7s 46ms/step - loss: 0.0553 - val_loss: 1.5759
Epoch 162/400
144/144 [==============================] - 6s 41ms/step - loss: 0.0547 - val_loss: 1.5751
Epoch 163/400
144/144 [==============================] - 6s 42ms/step - loss: 0.0550 - val_loss: 1.6132
Epoch 164/400
144/144 [==============================] - 6s 42ms/step - loss: 0.0548 - val_loss: 1.5863
Epoch 165/400
144/144 [==============================] - 6s 44ms/step - lo

144/144 [==============================] - 9s 63ms/step - loss: 0.0380 - val_loss: 1.7567
Epoch 235/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0392 - val_loss: 1.7219
Epoch 236/400
144/144 [==============================] - 9s 62ms/step - loss: 0.0393 - val_loss: 1.7200
Epoch 237/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0398 - val_loss: 1.7022
Epoch 238/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0400 - val_loss: 1.7318
Epoch 239/400
144/144 [==============================] - 9s 61ms/step - loss: 0.0405 - val_loss: 1.7400
Epoch 240/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0402 - val_loss: 1.7102
Epoch 241/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0403 - val_loss: 1.7260
Epoch 242/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0398 - val_loss: 1.7059
Epoch 243/400
144/144 [==============================] - 9s 62ms/step - loss: 

Epoch 313/400
144/144 [==============================] - 9s 64ms/step - loss: 0.0313 - val_loss: 1.8559
Epoch 314/400
144/144 [==============================] - 9s 62ms/step - loss: 0.0343 - val_loss: 1.8608
Epoch 315/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0335 - val_loss: 1.8615
Epoch 316/400
144/144 [==============================] - 9s 62ms/step - loss: 0.0297 - val_loss: 1.8364
Epoch 317/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0314 - val_loss: 1.8585
Epoch 318/400
144/144 [==============================] - 9s 62ms/step - loss: 0.0302 - val_loss: 1.8687
Epoch 319/400
144/144 [==============================] - 9s 62ms/step - loss: 0.0322 - val_loss: 1.9020
Epoch 320/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0310 - val_loss: 1.8516
Epoch 321/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0335 - val_loss: 1.8821
Epoch 322/400
144/144 [==============================] - 9s 64ms

144/144 [==============================] - 9s 63ms/step - loss: 0.0278 - val_loss: 1.9119
Epoch 392/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0253 - val_loss: 1.9380
Epoch 393/400
144/144 [==============================] - 9s 62ms/step - loss: 0.0257 - val_loss: 1.9166
Epoch 394/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0264 - val_loss: 1.9315
Epoch 395/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0247 - val_loss: 1.9192
Epoch 396/400
144/144 [==============================] - 9s 62ms/step - loss: 0.0267 - val_loss: 1.9441
Epoch 397/400
144/144 [==============================] - 9s 62ms/step - loss: 0.0255 - val_loss: 1.9456
Epoch 398/400
144/144 [==============================] - 9s 63ms/step - loss: 0.0274 - val_loss: 1.9425
Epoch 399/400
144/144 [==============================] - 9s 64ms/step - loss: 0.0255 - val_loss: 1.9276
Epoch 400/400
144/144 [==============================] - 9s 63ms/step - loss: 

/home/kritz/anaconda3/envs/scripts/lib/python3.6/site-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


-
Input sentence: Calm down.
Decoded sentence: என்னுடைய கணக்கிற்கு மாற்று

-
Input sentence: Who is he?
Decoded sentence: நாங்கள் திருடனுக்குப் பின்னால் ஓடினோம்

-
Input sentence: Who knows?
Decoded sentence: அவள் யார்?

-
Input sentence: She smiled.
Decoded sentence: அவள் அன்பானவள்

-
Input sentence: Talk to me!
Decoded sentence: என்னிடம் பேசு

-
Input sentence: Who is she?
Decoded sentence: நாங்கள் திருடனுக்குப் பின்னால் ஓடினோம்

-
Input sentence: Go to sleep.
Decoded sentence: அவள் உன்னை வெறுக்கிற மாதிரி தெரிகிறது

-
Input sentence: It may rain.
Decoded sentence: நான் வெளி நாட்டிற்குச் செல்ல விரும்புகிறேன்

-
Input sentence: She bit him.
Decoded sentence: அவள் அவனைக் கடித்தாள்

-
Input sentence: She hit him.
Decoded sentence: அவள் அன்பானவள்

-
Input sentence: She is kind.
Decoded sentence: அவள் அன்பானவள்

-
Input sentence: She is eight.
Decoded sentence: அவள் அன்பானவள்

-
Input sentence: Where are we?
Decoded sentence: அவர்கள் மேரியை கிண்டலடித்தார்கள்

-
Input sentence: Keep in touc

-
Input sentence: All of us were silent.
Decoded sentence: அவர் உங்களுடைய நண்பரா?

-
Input sentence: Be kind to old people.
Decoded sentence: வயோதிகர்களிடம் அன்பாக இரு

-
Input sentence: Beware of pickpockets.
Decoded sentence: ஜேப்படிகாரர்களிடம் ஜாக்கிரதையாக இருக்கவும்

-
Input sentence: Don't drink and drive.
Decoded sentence: அந்த பக்கம்தான் வழி



In [8]:
seq_index = [4,10,15,21]
for each in seq_index:
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[each: each + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[each])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Talk to me!
Decoded sentence: என்னிடம் பேசு

-
Input sentence: She is kind.
Decoded sentence: அவள் அன்பானவள்

-
Input sentence: Give it to her.
Decoded sentence: அவளிடம் கொடு

-
Input sentence: That's the way.
Decoded sentence: அந்த பக்கம்தான் வழி

